# Creating an Einstein Toolkit thorn:
## The scalar wave equation

The goal of this tutorial is to construct an Einstein Toolkit (ETK) thorn that will evolve some initial data according to the scalar wave equation, subject to appropriate boundary conditions. This thorn should function identically to the $\text{WaveToy}$ thorns included in the ETK.

After importing the core modules, we will set $\text{GridFuncMemAccess}$ to $\text{ETK}$. The scalar wave right-hand sides and plane wave initial data are already built by [$\text{Tutorial-ScalarWave.ipynb}$](Tutorial-ScalarWave.ipynb), so we can simply import them to use here. We will also need to instruct the system to create the directories for our thorn.

In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

par.set_parval_from_str("grid::GridFuncMemAccess","ETK")

# Step 1b: Call the ScalarWave_RHSs() function from within the
#         ScalarWave/ScalarWave_RHSs.py module,
#         which should do exactly the same as in Steps 1-10 above.
import ScalarWave.ScalarWave_RHSs as swrhs
swrhs.ScalarWave_RHSs()

# Step 1c: Call the InitialData_PlaneWave() function from within the
#         ScalarWave/InitialData_PlaneWave.py module,
#         which should do exactly the same as in Steps 1-5 above.
import ScalarWave.InitialData_PlaneWave as swid
swid.InitialData_PlaneWave()

!mkdir ScalarWave     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir ScalarWave/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.


Now, we need to output these expressions to C code. We will do this the same way as in the Weyl scalars ETK tutorial: export these to a .h file, which can then be included in a main .c file to do the calculations.

In [2]:
# First, we will write the right-hand sides.
scalar_RHSs_to_print = [\
                        lhrh(lhs=gri.gfaccess("out_gfs","uu"),rhs=swrhs.uu_rhs),\
                        lhrh(lhs=gri.gfaccess("out_gfs","vv"),rhs=swrhs.vv_rhs),]
scalar_RHSs_CcodeKernel = fin.FD_outputC("returnstring",scalar_RHSs_to_print)

scalar_RHSs_looped = loop.loop(["i2","i1","i0"],["1","1","1"],["cctk_lsh[2]-1","cctk_lsh[1]-1","cctk_lsh[0]-1"],\
                               ["1","1","1"],["#pragma omp parallel for","",""],"",scalar_RHSs_CcodeKernel)

with open("ScalarWave/src/ScalarWave_RHSs.h", "w") as file:
    file.write(str(scalar_RHSs_looped))

# Second, we will write the initial data.
scalar_PWID_to_print = [\
                        lhrh(lhs=gri.gfaccess("out_gfs","uu"),rhs=swid.uu_ID),\
                        lhrh(lhs=gri.gfaccess("out_gfs","vv"),rhs=swid.vv_ID),]
scalar_PWID_CcodeKernel = fin.FD_outputC("returnstring",scalar_PWID_to_print)

scalar_PWID_looped = loop.loop(["i2","i1","i0"],["1","1","1"],["cctk_lsh[2]-1","cctk_lsh[1]-1","cctk_lsh[0]-1"],\
                               ["1","1","1"],["#pragma omp parallel for","",""],"",scalar_PWID_CcodeKernel)

with open("ScalarWave/src/ScalarWave_PWID.h", "w") as file:
    file.write(str(scalar_PWID_looped))
